#### Analyze RCS csv logs

In [1]:
import pandas as pd
import numpy as np
import math
import sys

In [4]:
file_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\logs\\202210280009-UbuntuXenial\\202210280009-UbuntuXenial.rcg.csv"
df = pd.read_csv(file_path, index_col=False, low_memory=False)
df = df.dropna(axis=1) # drop empty columns
# # add column to identify the current keeper with ball
# df['current_keeper'] = ''
# # add column to keep track of the current action
# df['current_action'] = ''

In [5]:
print(df.columns)

Index(['show_time', 'playmode', 'team_name_l', 'team_name_r', 'team_score_l',
       'team_score_r', 'team_pen_score_l', 'team_pen_score_r',
       'team_pen_miss_l', 'team_pen_miss_r',
       ...
       'player_l5_counting_dash', 'player_l5_counting_turn',
       'player_l5_counting_catch', 'player_l5_counting_move',
       'player_l5_counting_turn_neck', 'player_l5_counting_change_view',
       'player_l5_counting_say', 'player_l5_counting_tackle',
       'player_l5_counting_point_to', 'player_l5_counting_attention_to'],
      dtype='object', length=149)


In [ ]:
# center point
center_x = 0
center_y = 0
# ball kickable distance = player_size + ball_size + kickable margin
ball_kickable_dist = 0.03 + 0.085 + 0.7
# variables to identify data columns
# 3v2
keeper_col_name_prefixes = ['player_l' + str(i) for i in range(1, 4)]
taker_col_name_prefixes = ['player_l' + str(i) for i in range(4, 6)]
keeper_ids = [i for i in range(1, 4)]
taker_ids = [i for i in range(4, 6)]
actions_3v2 = {
    0: 'Hold',
    1: 'Pass to K1',
    2: 'Pass to K2'
}
# add 3v2 specific state columns
state_3v2_col_names = ['dist(K1,C)', 'dist(K1,K2)', 'dist(K1,K3)', 'dist(K1,T1)', 'dist(K1,T2)',
                        'dist(K2,C)', 'dist(K3,C)', 'dist(T1,C)', 'dist(T2,C)', 'Min(dist(K2,T1),dist(K2,T2))',
                        'Min(dist(K3,T1),dist(K3,T2))', 'Min(ang(K2,K1,T1),ang(K2,K1,T2))', 'Min(ang(K3,K1,T1),ang(K3,K1,T2))']
for col in state_3v2_col_names:
    df[col] = ''

the corners of the keepaway box are defined as corner = (length / 2, weidth / 2)

for most of the experiments length = width = 20

thus, the corners are (10, 10) and (-10, -10)

this implies that the center is (0, 0)

0x3 seems to denote that the ball is kickable
0x441 is collision with ball
0x801 also seems to denote collision with ball
0x803 ball is kickable, no collison

In [ ]:
for col in df.columns:
    if 'player' in col and 'side' in col:
        print(col)

In [ ]:
pd.unique(df['player_l1_state'])

In [ ]:
pd.unique(df['player_l2_state'])

#### 202210280159-UbuntuXenial analysis

In [9]:
file_path = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\logs\\202210280159-UbuntuXenial\\202210280159-UbuntuXenial.rcg.csv"
df = pd.read_csv(file_path, index_col=False, low_memory=False)
df = df.dropna(axis=1) # drop empty columns
# add column to identify the current keeper with ball
df['current_keeper'] = ''
# add column to keep track of the current action
df['current_action'] = ''

In [ ]:
# center point
center_x = 0
center_y = 0
# ball kickable distance = player_size + ball_size + kickable margin
ball_kickable_dist = 0.03 + 0.085 + 0.7
# variables to identify data columns
# 3v2
keeper_col_name_prefixes = ['player_l' + str(i) for i in range(1, 4)]
taker_col_name_prefixes = ['player_l' + str(i) for i in range(4, 6)]
keeper_ids = [i for i in range(1, 4)]
taker_ids = [i for i in range(4, 6)]
actions_3v2 = {
    0: 'Hold',
    1: 'Pass to K1',
    2: 'Pass to K2'
}
# add 3v2 specific state columns
state_3v2_col_names = ['dist(K1,C)', 'dist(K1,K2)', 'dist(K1,K3)', 'dist(K1,T1)', 'dist(K1,T2)',
                        'dist(K2,C)', 'dist(K3,C)', 'dist(T1,C)', 'dist(T2,C)', 'Min(dist(K2,T1),dist(K2,T2))',
                        'Min(dist(K3,T1),dist(K3,T2))', 'Min(ang(K2,K1,T1),ang(K2,K1,T2))', 'Min(ang(K3,K1,T1),ang(K3,K1,T2))']
for col in state_3v2_col_names:
    df[col] = ''

In [ ]:
def get_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [ ]:
def get_closest_player_to_ball(player_positions:list, ball_location:tuple) -> int:
    ball_x, ball_y = ball_location
    closest_distance = sys.maxsize * 2 + 1
    closest_player_id = -1
    for player_idx, player_pos in enumerate(player_positions):
        player_x = player_pos[0]
        player_y = player_pos[1]
        distance_to_ball = get_distance(player_x, player_y, ball_x, ball_y)
        if distance_to_ball < closest_distance:
            closest_distance = distance_to_ball
            closest_player_id = player_idx + 1
    return closest_player_id, closest_distance

In [ ]:
def get_sorted_distance_between_current_and_others(current_player_location:tuple, other_positions:list):
    current_player_x, current_player_y = current_player_location
    distances = [get_distance(current_player_x, current_player_y, player_x, player_y) for player_x, player_y in other_positions if (player_x, player_y) != current_player_location]
    return sorted(distances)

In [ ]:
def get_sorted_teammates_idx_to_cur_player(cur_player_id:int, player_positions:list):
    assert cur_player_id > 0
    cur_player_idx = cur_player_id - 1
    cur_player_x, cur_player_y = player_positions[cur_player_idx][0], player_positions[cur_player_idx][1]
    distances = [get_distance(cur_player_x, cur_player_y, player_x, player_y) for player_x, player_y in player_positions]
    return np.argsort(distances)[1:] # drop the first index as it counts the current keeper

In [ ]:
def get_sorted_opponents_idx_to_cur_player(cur_player_pos:tuple, opponent_positions:list):
    cur_player_x, cur_player_y = cur_player_pos
    distances = [get_distance(cur_player_x, cur_player_y, opp_x, opp_y) for opp_x, opp_y in opponent_positions]
    return np.argsort(distances)

In [ ]:
def get_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

In [ ]:
show_time = 0
samples = 0
# get a list of ids for the keepers
for index, row in df.iterrows():
    # get the keeper and taker positions
    keeper_positions = [(row['player_l{}_x'.format(id)], row['player_l{}_y'.format(id)]) for id in keeper_ids]
    taker_positions = [(row['player_l{}_x'.format(id)], row['player_l{}_y'.format(id)]) for id in taker_ids]
    # get ball position
    ball_x = row['ball_x']
    ball_y = row['ball_y']
    # get closest keeper to ball
    closest_keeper_id, closest_distance_to_ball = get_closest_player_to_ball(keeper_positions, (ball_x, ball_y))
    ball_velocity = math.sqrt(row['ball_vx']**2 + row['ball_vy']**2)
    if closest_distance_to_ball < ball_kickable_dist and ball_velocity <= 2: # we only fill information if a keeper actually has the ball
        # get a list of teammate idxes sorted according to distance to the current keeper
        sorted_teammate_idxes = get_sorted_teammates_idx_to_cur_player(closest_keeper_id, keeper_positions)
        # get a list of taker idxes sorted according to distance to the current keeper
        sorted_opponent_idxes = get_sorted_opponents_idx_to_cur_player(keeper_positions[closest_keeper_id - 1], taker_positions)
        # distance between current keeper to the center of the field
        keeper_with_ball_x = keeper_positions[closest_keeper_id - 1][0]
        keeper_with_ball_y = keeper_positions[closest_keeper_id - 1][1]
        df.at[index, 'dist(K1,C)'] = get_distance(keeper_with_ball_x, keeper_with_ball_y, center_x, center_y)
        # distance between current keeper and the teammates, sorted according to distance
        for counter, teammate_idx in enumerate(sorted_teammate_idxes):
            teammate_x, teammate_y = keeper_positions[teammate_idx][0], keeper_positions[teammate_idx][1]
            df.at[index, 'dist(K1,K{})'.format(counter + 2)] = get_distance(keeper_with_ball_x, keeper_with_ball_y, teammate_x, teammate_y)
            # distance between teammates to the center
            df.at[index, 'dist(K{},C)'.format(counter + 2)] = get_distance(teammate_x, teammate_y, center_x, center_y)
        # distance between current keeper and takers, sorted according to distance
        for counter, opp_idx in enumerate(sorted_opponent_idxes):
            opp_x, opp_y = taker_positions[opp_idx][0], taker_positions[opp_idx][1]
            df.at[index, 'dist(K1,T{})'.format(counter + 1)] = get_distance(keeper_with_ball_x, keeper_with_ball_y, opp_x, opp_y)
            # distance between opponent to the center
            df.at[index, 'dist(T{},C)'.format(counter + 1)] = get_distance(opp_x, opp_y, center_x, center_y)
        # distance between teammates and opponents
        for tm_counter, teammate_idx in enumerate(sorted_teammate_idxes):
            tm_x, tm_y = keeper_positions[teammate_idx][0], keeper_positions[teammate_idx][1]
            min_dist = get_sorted_distance_between_current_and_others((tm_x, tm_y), taker_positions)[0]
            col_name = ''
            for opp_counter, opp_idx in enumerate(sorted_opponent_idxes):
                col_name += 'dist(K{},T{}),'.format(tm_counter + 2, opp_counter + 1)
            col_name = col_name[:-1]
            df.at[index, 'Min({})'.format(col_name)] = min_dist
            # angle of passing lane from keeper with ball to teammates
            min_angle = sys.maxsize * 2 + 1
            col_name = ''
            for opp_counter, opp_idx in enumerate(sorted_opponent_idxes):
                angle = get_angle((tm_x, tm_y), (keeper_with_ball_x, keeper_with_ball_y), taker_positions[opp_idx])
                if angle < min_angle:
                    min_angle = angle
                col_name += 'ang(K{},K1,T{}),'.format(tm_counter + 2, opp_counter + 1)
            col_name = col_name[:-1]
            df.at[index, 'Min({})'.format(col_name)] = min_angle
        # debug print
        print('Time: {}'.format(show_time))
        for keeper_idx, keeper_pos in enumerate(keeper_positions):
            print('Keeper {} , Keeper x: {}, Keeper y: {}, distance to ball: {}'.format(keeper_idx + 1, keeper_pos[0], keeper_pos[1], get_distance(keeper_pos[0], keeper_pos[1], ball_x, ball_y)))
        for taker_idx, taker_pos in enumerate(taker_positions):
            print('Taker {}, Taker x: {}, Taker y: {}, distance to ball: {}'.format(taker_idx + 1, taker_pos[0], taker_pos[1], get_distance(taker_pos[0], taker_pos[1], ball_x, ball_y)))
        for state_var in state_3v2_col_names:
            print('{}: {}'.format(state_var, row[state_var]))
        print('Ball velocity: {}'.format(ball_velocity))
        print('\n')
        samples += 1
    show_time += 1
print('Total samples: {}'.format(samples))

In [ ]:
from IPython.display import display, HTML
display(df)

In [ ]:
show_time = 0
for index, row in df.iterrows():
    print('Time: {}'.format(show_time))
    show_time += 1
    ball_x = row['ball_x']
    ball_y = row['ball_y']
    keeper_positions = []
    # print keeper distance to ball
    for keeper_idx, keeper in enumerate(keepers_names):
        keeper_x = row[keeper + '_x']
        keeper_y = row[keeper + '_y']
        print('Keeper {} , Keeper x: {}, Keeper y: {}, distance to ball: {}'.format(keeper_idx + 1, keeper_x, keeper_y, get_distance(keeper_x, keeper_y, ball_x, ball_y)))
        keeper_positions.append((keeper_x, keeper_y))
    closest_keeper, closest_distance_to_ball = get_closest_player_to_ball(keeper_positions, (ball_x, ball_y))
    if closest_distance_to_ball < ball_kickable_dist:
        print('Keeper {} can kick the ball.'.format(closest_keeper + 1))
        # start creating state variables in relation to the keeper with the ball
        keeper_with_ball_x = row['player_l{}_x'.format(closest_keeper + 1)]
        keeper_with_ball_y = row['player_l{}_y'.format(closest_keeper + 1)]
        # distance between current keeper to the center of the field
        row['dist(K1,C)'] = get_distance(keeper_with_ball_x, keeper_with_ball_y, center_x, center_y)
        # distance between current keeper to teammates
        distance_between_keeper_and_teammates = get_sorted_distance_between_player_and_teammates(keeper_positions, (keeper_with_ball_x, keeper_with_ball_y))
        for d_idx, d in enumerate(distance_between_keeper_and_teammates):
            row['dist(K1,K{})'.format(d_idx + 2)] = d
        # distance between teammates and the center of the field
        
    # print taker distance to ball
    for taker_idx, taker in enumerate(takers_names):
        taker_x = row[taker + '_x']
        taker_y = row[taker + '_y']
        print('Taker {} distance to ball: {}'.format(taker_idx + 1, get_distance(taker_x, taker_y, ball_x, ball_y)))
    print('\n')

In [ ]:
def get_closest_player_to_ball(player_positions:list, ball_location:tuple) -> int:
    ball_x, ball_y = ball_location
    closest_distance = sys.maxsize * 2 + 1
    closest_player_id = -1
    for player_id, player_pos in enumerate(player_positions):
        player_x = player_pos[0]
        player_y = player_pos[1]
        distance_to_ball = get_distance(player_x, player_y, ball_x, ball_y)
        if distance_to_ball < closest_distance:
            closest_distance = distance_to_ball
            closest_player_id = player_id
    return closest_player_id, closest_distance

In [36]:
data_column_names = ['Cycle', 'State']
data_column_dtypes = ['int', 'str']
data_column_info = {col_name : col_dtype for col_name, col_dtype in zip(data_column_names, data_column_dtypes)}
df2 = pd.DataFrame({c: pd.Series(dtype=dt) for c, dt in data_column_info.items()})
filepath = "C:\\Users\\minhh\\Documents\\JHU\\Fall 2022\\Evolutionary and Swarm Intelligence\\src\\GAME\\logs\\202210280159-UbuntuXenial\\k3-draw.log"
# create an array to help us id the keeper
id_of_targets = []
with open(filepath, 'r') as f:
    lines = f.readlines()
    for line in lines:
        line_split = line.split(' ')
        if len(line_split) > 1 and line_split[1] == '\"state\"':
            cycle = int(line_split[0][:-1])
            state = line.split('\"')[3]
            if '(' in state:
                # run a check to help us id the current keeper
                id_of_target_kp = state[1:-1].split(' ')[2]
                if id_of_target_kp not in id_of_targets:
                    id_of_targets.append(id_of_target_kp)
            df2 = df2.append({'Cycle': cycle, 'State': state}, ignore_index=True)

In [ ]:
df2

In [37]:
time_of_first_action = -1
for _, row in df2.iterrows():
    if '(' in row['State']:
        time_of_first_action = row['Cycle'] + 1
        break

In [38]:
time_of_first_action

41

In [39]:
keeper_ids = [i for i in range(1, 3 + 1)]
rcg_at_time_slice = df[df['show_time'] == time_of_first_action]
for col in rcg_at_time_slice.columns:
    if rcg_at_time_slice[col].to_list()[0] == '0x3':
        print(col)

player_l2_state


In [42]:
'player_l2_state'.split('_')[1][1:]

'2'

In [ ]:
def id_keeper_draw_log()